### Connect to Gremlin Server

In [1]:
import sys
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)


from gremlin_python import statics
from gremlin_python.process.traversal import T, Direction
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import GraphTraversalSource
from gremlin_python.process.graph_traversal import __

from graph.base import g
from ipycytoscape_graph_visualization import visualize_graph

from dotenv import load_dotenv

load_dotenv()

# test connection to gremlin server
g.V().limit(1).toList()

[v[4112]]

### Analyze Address `1BBZ`

In [2]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
# interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)
# graph = analyzer.traversal_to_networkx(my_hist)

print(my_hist)
print(graph)

# dump graph to gexf file
# addr_hist_graph_path = Path('/', 'app', 'addr_hist_graph.gexf')
# nx.write_gexf(graph, addr_hist_graph_path)

coin_sources = analyzer.get_coin_traces(address.id, 'address', direction='incoming', graph=graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {round(source['amount'], 10)}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

id of address 1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs: 504
[['withStrategies', OptionsStrategy], ['withStrategies', OptionsStrategy]][['V'], ['has', 'address_id', 504], ['repeat', [['inE', 'sent'], ['otherV']]], ['emit'], ['path'], ['by', [['elementMap']]], ['by', [['elementMap']]], ['unfold']]
DiGraph with 10 nodes and 10 edges
amount from 187:1:0 15NU (1.00000000) is 1.0
amount from 183:1:0 13Ht (1.00000000) is 1.0
amount from 182:1:1 12cb (29.00000000) is 11.0
amount from 181:1:1 12cb (30.00000000) is 11.0
amount from 170:1:1 12cb (40.00000000) is 11.0
amount from 9:0:0 12cb (50.00000000) is 11.0
amount from 248:1:0 1ByL (10.00000000) is 10.0
amount from 183:1:1 12cb (28.00000000) is 10.0
amount from 360:0:0 18SH (50.00000000) is 50.0


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Apply Manual Proportions and Visualize the Transaction History, then Reset the Proportions

In [3]:
import networkx as nx
from sqlalchemy.orm import joinedload

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output, ManualProportion
from graph.base import g
from graph_populate import PopulateOutputProportionGraph
from graph_analyze import GraphAnalyzer

graph_populator = PopulateOutputProportionGraph(SessionLocal)
analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
# interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'


with SessionLocal() as session:
    # apply manual proportion to first input/output pair in >= second tx of block 546
    tx_list = session.query(Tx)\
                     .options(
                         joinedload(Tx.inputs).joinedload(Input.prev_out),
                         joinedload(Tx.outputs)
                     )\
                     .filter(Tx.index_in_block >= 1, Tx.block_height == 546)\
                     .order_by(Tx.index_in_block)\
                     .all()
    tx_input_ids = [input.id for tx in tx_list for input in tx.inputs]
    # create the manual proportion object
    manual_proportions = session.query(ManualProportion)\
                                .filter(ManualProportion.input_id.in_(tx_input_ids))\
                                .all()
    if len(manual_proportions) == 0:
        session.add_all([
            ManualProportion(
                input_id=tx_list[0].inputs[0].id,
                output_id=tx_list[0].outputs[0].id,
                proportion=1.0
            ),
            ManualProportion(
                input_id=tx_list[0].inputs[1].id,
                output_id=tx_list[0].outputs[1].id,
                proportion=1.0
            ),
            ManualProportion(
                input_id=tx_list[1].inputs[0].id,
                output_id=tx_list[1].outputs[0].id,
                proportion=1.0
            ),
            ManualProportion(
                input_id=tx_list[1].inputs[1].id,
                output_id=tx_list[1].outputs[1].id,
                proportion=1.0
            ),
            ManualProportion(
                input_id=tx_list[2].inputs[0].id,
                output_id=tx_list[2].outputs[0].id,
                proportion=1.0
            ),
            ManualProportion(
                input_id=tx_list[2].inputs[1].id,
                output_id=tx_list[2].outputs[1].id,
                proportion=1.0
            )
        ])
        session.commit()
    graph_populator.apply_manual_edge_proportions(session, show_progressbar=True)
    address = session.query(Address).filter_by(addr=interesting_addr).first()

if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)
print(graph)

coin_sources = analyzer.get_coin_traces(address.id, 'address', direction='incoming', graph=graph, pretty_labels=True)

for source in coin_sources.values():
    print(f"amount from {source['label']} is {round(source['amount'], 10)}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))

with SessionLocal() as session:
    graph_populator.reset_manual_edge_proportions(session, show_progressbar=True)
    items_deleted = session.query(ManualProportion)\
                                .filter(ManualProportion.input_id.in_(tx_input_ids))\
                                .delete()

    print(f"deleted {items_deleted} items")
    session.commit()

my_hist = analyzer.get_address_history(interesting_addr)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)
print(graph)
display(visualize_graph(graph, layout='dagre'))

Applying manual edge proportions:  50%|█████     | 3/6 [00:02<00:02,  1.07edge/s]

id of address 1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv: 557
DiGraph with 9 nodes and 8 edges
amount from 546:1:0 1KAD (1.00000000) is 0.04
amount from 545:1:0 1DZT (1.00000000) is 0.04
amount from 524:1:0 1DCb (25.00000000) is 23.08
amount from 286:0:0 1Jhk (50.00000000) is 23.08
amount from 546:2:1 1KAD (24.00000000) is 23.04
amount from 546:1:1 1DZT (24.00000000) is 23.04
amount from 545:1:1 1DCb (24.00000000) is 23.04


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

Resetting edge proportions: 100%|██████████| 3/3 [00:00<00:00, 19.74edge/s]

deleted 6 items
DiGraph with 9 nodes and 13 edges


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Trace Forwards from `1Jhk` TODO: Limit depth

In [ ]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

interesting_addr = '1Jhk2DHosaaZx1E4CbnTGcKM7FC88YHYv9'

with SessionLocal() as session:
    address = session.query(Address)\
                     .filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

# depth of 15 works
# depth of 30 is too much
# what about 20?
my_hist = analyzer.get_vertex_path(address.id, 'address', depth=16)
graph = analyzer.traversal_to_networkx(my_hist, include_data=True)

print(graph)

# save graph to file in docs/graphs folder
graph_path = Path('/', 'app', 'graph_file_exports', '1jkh_paths.gexf')
nx.write_gexf(graph, graph_path)

# coin_sources = analyzer.get_coin_traces(address.id,
#                                         'address',
#                                         direction='outgoing',
#                                         graph=graph,
#                                         pretty_labels=True)

# for source in list(coin_sources.values())[:5]:
#     print(f"amount from {source['label']} is {round(source['amount'], 10)}")

# from ipycytoscape_graph_visualization import visualize_graph

# display(visualize_graph(graph, layout='dagre'))

## Analyze Large Transactions

###  Get Vertices that are Common Between These Large Transactions

In [2]:
from collections import defaultdict
import itertools
import random

import networkx as nx
from rich.console import Console
from rich.table import Table

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output, BITCOIN_TO_SATOSHI
from graph_analyze import GraphAnalyzer


largest_txs = {
    '1c19': {
        'id': 951352,
        'depth': 10
    },
    'd508': {
        'id': 1035342,
        'depth': 10
    },
    '67e1': {
        'id': 1441873,
        'depth': 10
    },
    '41cb': {
        'id': 5575007,
        'depth': 4
    },
    '2f66': {
        'id': 7020591,
        'depth': 3
    },
    '06bc': {
        'id': 6987093,
        'depth': 3
    },
    '9e33': {
        'id': 1992599,
        'depth': 3
    },
    'b590': {
        'id': 5855968,
        'depth': 3
    },
    'e43d': {
        'id': 7186522,
        'depth': 3
    },
    '916e': {
        'id': 6336328,
        'depth': 3
    }
}

console = Console()
analyzer = GraphAnalyzer(g, SessionLocal)

# assign time values to the transactions
for tx_key, tx_info in largest_txs.items():
    with SessionLocal() as session:
        largest_txs[tx_key]['timestamp'] = session.query(Tx).filter_by(id=tx_info['id']).first().block.timestamp
        print(f"tx {tx_key} date: {largest_txs[tx_key]['timestamp'].date()}")

# Generate graphs for each transaction and track vertices
for tx_key, tx_info in largest_txs.items():
    with SessionLocal() as session:
        largest_txs[tx_key]['timestamp'] = session.query(Tx).filter_by(id=tx_info['id']).first().block.timestamp
        output = session.query(Output).filter_by(tx_id=tx_info['id']).first()
        history = analyzer.get_vertex_history(output.id, 'output', depth=largest_txs[tx_key]['depth'])
        tx_graph = analyzer.traversal_to_networkx(history, include_data=True)
        largest_txs[tx_key]['graph'] = tx_graph
        for vertex in tx_graph.nodes:
            output_id = tx_graph.nodes[vertex]['output_id']


def generate_random_color():
    return "#" + ''.join([random.choice('0123456789ABCDEF') for _ in range(6)])


def assign_vertex_colors(largest_txs, analyzer):
    tx_graphs = {}
    vertex_to_tx = defaultdict(set)
    all_tx_keys = set(largest_txs.keys())

    # Generate a mapping from vertices to the transactions they belong to
    for tx_key, tx_info in largest_txs.items():
        tx_graphs[tx_key] = tx_info['graph']
        for vertex in tx_info['graph'].nodes:
            output_id = tx_info['graph'].nodes[vertex]['output_id']
            vertex_to_tx[output_id].add(tx_key)

    # map from transaction subsets to colors and vertex counts
    subset_to_color = {}
    subset_to_vertex_count = defaultdict(int)

    # Predefined colors for different combinations
    predefined_colors = ['blue', 'green', 'yellow', 'orange', 'purple', 'cyan', 'magenta', 'brown', 'pink', 'teal']
    color_iterator = iter(predefined_colors)

    subset_to_color[tuple(sorted(all_tx_keys))] = 'red'

    # Generate all non-singleton subsets of transactions
    for r in range(2, len(all_tx_keys)):
        for subset in itertools.combinations(all_tx_keys, r):
            subset = tuple(sorted(subset))
            try:
                subset_to_color[subset] = next(color_iterator)
            except StopIteration:
                subset_to_color[subset] = generate_random_color()  # Generate a random color if we run out
            
            print(f"subset {subset} has color {subset_to_color[subset]}")

    # Assign colors to vertices based on their transaction groupings
    # and count number of vertices in each subset
    vertex_colors = {}
    for vertex, tx_keys in vertex_to_tx.items():
        subset_key = tuple(sorted(tx_keys))
        # Increment count for this subset
        subset_to_vertex_count[subset_key] += 1
        if len(tx_keys) == 1:
            vertex_colors[vertex] = 'grey'  # Default color for vertices unique to one transaction
        else:
            assert subset_key in subset_to_color, f"subset_key {subset_key} not found in subset_to_color"
            vertex_colors[vertex] = subset_to_color[subset_key]

    # Apply colors to the graphs
    node_color_maps = {}
    for tx_key, tx_graph in tx_graphs.items():
        node_color_map = {}
        for vertex in tx_graph.nodes:
            output_id = tx_graph.nodes[vertex]['output_id']
            node_color_map[vertex] = vertex_colors[output_id]
        node_color_maps[tx_key] = node_color_map

    # Return the graphs with the color mappings
    return node_color_maps, subset_to_vertex_count

def print_input_time_distribution(session, tx):
    # find the times that the inputs span
    input_times = [input.prev_out.transaction.block.timestamp for input in tx.inputs]
    print(f"input times: {input_times}")
    input_data = [
        {'time': input.prev_out.transaction.block.timestamp,
         'height': input.prev_out.transaction.block.height,
         'tx_hash': input.prev_out.transaction.hash,
         'value': input.prev_out.value / BITCOIN_TO_SATOSHI,
         'is_coinbase': input.prev_out.transaction.is_coinbase()}
         for input in tx.inputs]


    # find a grouping of two inputs which have the same block height
    # and print them out
    from collections import defaultdict
    same_time_inputs = defaultdict(list)
    for input in input_data:
        same_time_inputs[input['time']].append(input)

    for time, inputs in same_time_inputs.items():
        if len(inputs) > 1:
            print(f"inputs at time {time}:")
            for input in inputs:
                print(input)

    # Organize inputs by their reception time
    inputs_by_time = defaultdict(list)
    for input in input_data:
        inputs_by_time[input['time']].append(input)

    # Create a table
    table = Table(show_header=True, header_style="bold magenta")
    table.add_column("Time", style="dim", width=12)
    table.add_column("Block" , style="dim", width=8)
    table.add_column("Tx" , style="dim", width=7)
    table.add_column("Value (BTC)", justify="right")
    table.add_column("Coinbase", justify="center")

    for time, inputs_at_time in sorted(inputs_by_time.items()):
        # Check if multiple inputs were received at the same time
        if len(inputs_at_time) > 1:
            for input in inputs_at_time:
                table.add_row(
                    str(time.date()),
                    str(input['height']),
                    f"{input['tx_hash']}",
                    f"{input['value']:.8f}",
                    "Yes" if input['is_coinbase'] else "No"
                )
        else:
            input = inputs_at_time[0]
            table.add_row(
                str(time.date()),
                str(input['height']),
                f"{input['tx_hash']}",
                f"{input['value']:.8f}",
                "Yes" if input['is_coinbase'] else "No"
            )

    # Print the table
    console.print(table)

# Usage example
node_color_maps, subset_to_vertex_count = assign_vertex_colors(largest_txs, analyzer)

for tx_key, node_color_map in node_color_maps.items():
    largest_txs[tx_key]['color_map'] = node_color_map


tx 1c19 date: 2011-07-05
tx d508 date: 2011-07-14
tx 67e1 date: 2011-09-04
tx 41cb date: 2012-08-03
tx 2f66 date: 2012-09-12
tx 06bc date: 2012-09-11
tx 9e33 date: 2011-12-08
tx b590 date: 2012-08-12
tx e43d date: 2012-09-18
tx 916e date: 2012-08-24
subset ('41cb', 'b590') has color blue
subset ('41cb', '67e1') has color green
subset ('1c19', '41cb') has color yellow
subset ('41cb', '9e33') has color orange
subset ('06bc', '41cb') has color purple
subset ('41cb', '916e') has color cyan
subset ('2f66', '41cb') has color magenta
subset ('41cb', 'd508') has color brown
subset ('41cb', 'e43d') has color pink
subset ('67e1', 'b590') has color teal
subset ('1c19', 'b590') has color #C4C356
subset ('9e33', 'b590') has color #A0BC24
subset ('06bc', 'b590') has color #73F1F3
subset ('916e', 'b590') has color #8CFA8E
subset ('2f66', 'b590') has color #2D5EC1
subset ('b590', 'd508') has color #C16081
subset ('b590', 'e43d') has color #7678A8
subset ('1c19', '67e1') has color #EB7587
subset ('67e1

### Observe how the Large Transactions are Connected

Note that the transactions occurred on the following dates:
```
tx 1c19... date: 2011-07-05
tx d508... date: 2011-07-14
tx 67e1... date: 2011-09-04
tx 41cb... date: 2012-08-03
tx 2f66... date: 2012-09-12
tx 06bc... date: 2012-09-11
tx 9e33... date: 2011-12-08
tx b590... date: 2012-08-12
tx e43d... date: 2012-09-18
tx 916e... date: 2012-08-24
```

#### ('1c19', '67e1', 'd508')
```
tx 1c19... date: 2011-07-05
tx d508... date: 2011-07-14
tx 67e1... date: 2011-09-04
```

It is interesting to look at which transactions are connected. Many of these large transactions seem to be related to each other. Based on the limited of the graphs of these transaction histories, it was shown that subset ('1c19', '67e1', 'd508') has 421 vertices in common. These three transactions are largest in the dataset. And since they all occur within a few months of each other, it is likely that they are all part of the same activities.

In the cells below, each of these transactions' histories are visualized, and the results are very strange. First, `1c19...` receives large amounts of coin from mining, often in extremely small amounts such as 0.00000001 BTC (the smallest possible amount of bitcoin). They receive most of their coin from other large transactions, who themselves have extremely large and complex histories. `1c19...` has a total value of 455.48 BTC, and they send about 11.13 BTC to `d508...` (whose input value is 340.87444721). No bitcoin is sent directly from `d508...` to `67e1...`, but value was sent through one or more intermediate transactions. This pattern of splitting up large amounts of coin seems to be characteristic of a peeling chain, which is a common method used by criminals to launder money.

#### ('2f66', '41cb', '916e', 'b590', 'e43d')
```
tx 41cb... date: 2012-08-03
tx b590... date: 2012-08-12
tx 916e... date: 2012-08-24
tx e43d... date: 2012-09-18
tx 2f66... date: 2012-09-12
```

It is also observed that subset ('2f66', '41cb', '916e', 'b590', 'e43d') has 32 vertices in common. And they occured within the same one and a half month period.

In [17]:
print(sorted([len(key) for key in subset_to_vertex_count.keys()]))

for subset, count in subset_to_vertex_count.items():
    if len(subset) >= 3:
        print(f"subset {subset} has {count} vertices")


def find_common_outputs(session, earlier_tx_id, later_tx_id):
    return session.query(Output)\
                  .filter(Output.tx_id == earlier_tx_id)\
                  .join(Input, Output.id == Input.prev_out_id)\
                  .filter(Input.tx_id == later_tx_id)\
                  .all()

with SessionLocal() as session:
    
    print(f"inputs_d508_outputs_1c19: {[out.pretty_label() for out in find_common_outputs(
        session,
        largest_txs['1c19']['id'],
        largest_txs['d508']['id']
    )]}")

    print(f"inputs_67e1_outputs_d508: {[out.pretty_label() for out in find_common_outputs(
        session,
        largest_txs['d508']['id'],
        largest_txs['67e1']['id']
    )]}")

    print(f"inputs_67e1_outputs_1c19: {[out.pretty_label() for out in find_common_outputs(
        session,
        largest_txs['1c19']['id'],
        largest_txs['67e1']['id']
    )]}")

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 5]
subset ('1c19', '67e1', 'd508') has 421 vertices
subset ('2f66', '41cb', '916e', 'b590', 'e43d') has 32 vertices
subset ('41cb', '916e', 'b590') has 420 vertices
subset ('2f66', '41cb', '916e', 'e43d') has 8 vertices
subset ('2f66', '916e', 'b590', 'e43d') has 240 vertices
subset ('2f66', '916e', 'e43d') has 288 vertices
subset ('2f66', '916e', 'b590') has 86 vertices
subset ('06bc', '2f66', 'e43d') has 1 vertices
subset ('916e', 'b590', 'e43d') has 31 vertices
inputs_d508_outputs_1c19: ['134863:1:998 1Bjf (11.12963614)']
inputs_67e1_outputs_d508: []
inputs_67e1_outputs_1c19: []


### Traverse from First Output in Transaction `1c19...` with ID `951352` (Largest Tx from Blocks 0-200,000)

In [12]:
tx_key = '1c19'

display(visualize_graph(largest_txs[tx_key]['graph'], layout='dagre', node_color_map=largest_txs[tx_key]['color_map']))

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Further Analyze Transaction ID `951352`

In [13]:
from collections import defaultdict

from rich.console import Console
from rich.table import Table

from models.base import SessionLocal
from models.bitcoin_data import Address, Output, Input, Tx, BITCOIN_TO_SATOSHI
from blockchain_data_provider import PersistentBlockchainAPIData
from graph_analyze import GraphAnalyzer


data_provider = PersistentBlockchainAPIData()

tx_key = '1c19'

print(f"Transaction {tx_key} has {len(largest_txs[tx_key]['graph'].nodes)} vertices")
print(f"It occured on {largest_txs[tx_key]['timestamp']}")

with SessionLocal() as session:
    interesting_tx = data_provider.get_tx(
        session,
        largest_txs[tx_key]['id'],
    )
    print(f"hash of interesting_tx: {interesting_tx.hash}")
    min_value_input, min_value_output = min(interesting_tx.inputs, key=lambda x: x.prev_out.value), min(interesting_tx.outputs, key=lambda x: x.value)
    max_value_input, max_value_output = max(interesting_tx.inputs, key=lambda x: x.prev_out.value), max(interesting_tx.outputs, key=lambda x: x.value)
    second_highest_valued_input = sorted(interesting_tx.inputs, key=lambda x: x.prev_out.value)[-2]

    print(second_highest_valued_input.pretty_label())
    print(min_value_input.pretty_label())
    print(max_value_input.pretty_label())
    print(f"min_valued is coinbase? {min_value_input.prev_out.transaction.is_coinbase()}")

    print(f"Total tx input value: {sum([input.prev_out.value for input in interesting_tx.inputs]) / BITCOIN_TO_SATOSHI}")

    print_input_time_distribution(session, interesting_tx)


Transaction 1c19 has 743 vertices
It occured on 2011-07-05 11:41:00
hash of interesting_tx: 1c19389b0461f0901d8eace260764691926a5636c74bd8a3cc68db08dbbeb80a
134863:1:54 1FAW (32.58420199)
134863:1:0 13LF (0.00000001)
134863:1:99 1H1x (100.00000000)
min_valued is coinbase? True
Total tx input value: 455.47966241
input times: [datetime.datetime(2011, 6, 18, 15, 56, 38), datetime.datetime(2011, 5, 18, 19, 11, 48), datetime.datetime(2011, 6, 20, 6, 36, 58), datetime.datetime(2011, 6, 20, 20, 50, 13), datetime.datetime(2011, 6, 14, 14, 8, 22), datetime.datetime(2011, 6, 10, 7, 8, 12), datetime.datetime(2011, 6, 17, 1, 32, 51), datetime.datetime(2011, 6, 15, 22, 33, 8), datetime.datetime(2011, 6, 11, 9, 11, 11), datetime.datetime(2011, 6, 20, 23, 58, 36), datetime.datetime(2011, 6, 12, 9, 15, 51), datetime.datetime(2011, 6, 20, 15, 23, 38), datetime.datetime(2011, 6, 17, 12, 54, 39), datetime.datetime(2011, 6, 23, 7, 57, 41), datetime.datetime(2011, 6, 20, 20, 10, 28), datetime.datetime(2011

┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Time         ┃ Block    ┃  Value (BTC) ┃ Coinbase ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 2011-05-17   │ 124591   │   0.00000001 │   Yes    │
│ 2011-05-18   │ 124930   │   0.00000001 │   Yes    │
│ 2011-05-18   │ 124938   │   0.00000001 │   Yes    │
│ 2011-05-18   │ 124956   │   0.00000001 │   Yes    │
│ 2011-05-19   │ 125107   │   0.00000001 │   Yes    │
│ 2011-05-21   │ 125422   │   0.00000001 │   Yes    │
│ 2011-05-21   │ 125516   │   0.00000001 │   Yes    │
│ 2011-05-21   │ 125606   │   0.00000001 │   Yes    │
│ 2011-05-22   │ 125780   │   0.00000001 │   Yes    │
│ 2011-06-10   │ 129761   │  17.36446615 │   Yes    │
│ 2011-06-10   │ 129877   │  13.98103294 │   Yes    │
│ 2011-06-11   │ 130000   │   2.74611724 │   Yes    │
│ 2011-06-11   │ 130061   │   8.03580242 │   Yes    │
│ 2011-06-11   │ 130137   │  10.41399207 │   Yes    │
│ 2011-06-12   │ 130246   │   0.92378534 │   Yes    │
│ 2011-06-12   │ 130303   │  10.38531303 │   Yes    │
│ 2011-06-14   │ 130635   │   4.79176681 │   Yes    │
│ 2011-06-14   │ 130760   │   9.85168538 │   Yes    │
│ 2011-06-14   │ 130802   │   7.19264773 │   Yes    │
│ 2011-06-14   │ 130869   │  32.58420199 │   Yes    │
│ 2011-06-15   │ 130907   │  23.61581287 │   Yes    │
│ 2011-06-15   │ 130908   │  13.66915977 │   Yes    │
│ 2011-06-15   │ 130935   │  14.69633055 │   Yes    │
│ 2011-06-15   │ 130952   │  17.93922505 │   Yes    │
│ 2011-06-15   │ 130957   │   9.23855958 │   Yes    │
│ 2011-06-15   │ 130971   │  14.97746326 │   Yes    │
│ 2011-06-15   │ 130982   │   3.90326937 │   Yes    │
│ 2011-06-15   │ 131037   │   1.23424486 │   Yes    │
│ 2011-06-15   │ 131046   │  11.12524819 │   Yes    │
│ 2011-06-15   │ 131092   │   0.00000001 │   Yes    │
│ 2011-06-15   │ 131133   │   0.00000001 │   Yes    │
│ 2011-06-16   │ 131209   │   0.00000001 │   Yes    │
│ 2011-06-16   │ 131278   │   0.00000001 │   Yes    │
│ 2011-06-16   │ 131322   │   0.00000001 │   Yes    │
│ 2011-06-17   │ 131333   │   4.78931878 │   Yes    │
│ 2011-06-17   │ 131351   │   6.92522001 │   Yes    │
│ 2011-06-17   │ 131369   │  14.45802964 │   Yes    │
│ 2011-06-17   │ 131375   │   3.21981889 │   Yes    │
│ 2011-06-17   │ 131376   │   2.45645628 │   Yes    │
│ 2011-06-17   │ 131411   │   8.83375178 │   Yes    │
│ 2011-06-17   │ 131424   │   1.43019350 │   Yes    │
│ 2011-06-17   │ 131451   │   7.57090932 │   Yes    │
│ 2011-06-17   │ 131470   │   0.84952827 │   Yes    │
│ 2011-06-18   │ 131639   │   0.00000001 │   Yes    │
│ 2011-06-18   │ 131645   │   0.00000001 │   Yes    │
│ 2011-06-18   │ 131662   │   0.00000001 │   Yes    │
│ 2011-06-18   │ 131688   │   0.00000001 │   Yes    │
│ 2011-06-18   │ 131711   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131715   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131743   │   1.04574791 │   Yes    │
│ 2011-06-19   │ 131791   │  15.26525245 │   Yes    │
│ 2011-06-19   │ 131824   │   9.26798257 │   Yes    │
│ 2011-06-19   │ 131847   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131851   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131874   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131885   │   5.44116682 │   Yes    │
│ 2011-06-19   │ 131910   │   0.00000001 │   Yes    │
│ 2011-06-19   │ 131922   │   0.80677684 │   Yes    │
│ 2011-06-19   │ 131924   │   7.37419315 │   Yes    │
│ 2011-06-20   │ 131988   │   0.00000001 │   Yes    │
│ 2011-06-20   │ 131996   │   0.92238095 │   Yes    │
│ 2011-06-20   │ 131999   │   0.00000001 │   Yes    │
│ 2011-06-20   │ 132024   │   3.70988221 │   Yes    │
│ 2011-06-20   │ 132073   │   3.77894690 │   Yes    │
│ 2011-06-20   │ 132087   │   0.66915867 │   Yes    │
│ 2011-06-20   │ 132132   │   0.00000001 │   Yes    │
│ 2011-06-20   │ 132135   │   0.00000001 │   Yes    │
│ 2011-06-20   │ 132164   │   0.00000001 │   Yes    │
│ 2011-06-20   │ 132168   │   0.13981219 │   Yes    │
│ 2011-06-21   │ 132189   │   0.00000001 │   Yes    │
│ 2011-06-21   │ 132197   │   1.56214694 │   Yes    │
│ 20

### Analyze Transaction `d50871...` with ID `1035342` (2nd Largest Tx from Blocks 0-200,000)

In [4]:
tx_key = 'd508'

display(visualize_graph(largest_txs[tx_key]['graph'], layout='dagre', node_color_map=largest_txs[tx_key]['color_map']))

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…

### Further Analyze Transaction `d50871...` with ID `1035342`

In [8]:

from models.base import SessionLocal
from models.bitcoin_data import Address, Output, Input, Tx, BITCOIN_TO_SATOSHI
from blockchain_data_provider import PersistentBlockchainAPIData
from graph_analyze import GraphAnalyzer


data_provider = PersistentBlockchainAPIData()

tx_key = 'd508'

print(f"Transaction {tx_key} has {len(largest_txs[tx_key]['graph'].nodes)} vertices")
print(f"It occured on {largest_txs[tx_key]['timestamp']}")

with SessionLocal() as session:
    interesting_tx = data_provider.get_tx(
        session,
        largest_txs[tx_key]['id'],
    )
    print(f"hash of interesting_tx: {interesting_tx.hash}")
    min_value_input, min_value_output = min(interesting_tx.inputs, key=lambda x: x.prev_out.value), min(interesting_tx.outputs, key=lambda x: x.value)
    max_value_input, max_value_output = max(interesting_tx.inputs, key=lambda x: x.prev_out.value), max(interesting_tx.outputs, key=lambda x: x.value)
    second_highest_valued_input = sorted(interesting_tx.inputs, key=lambda x: x.prev_out.value)[-2]

    print(second_highest_valued_input.pretty_label())
    print(min_value_input.pretty_label())
    print(max_value_input.pretty_label())
    print(f"min_valued is coinbase? {min_value_input.prev_out.transaction.is_coinbase()}")

    print(f"Total tx input value: {sum([input.prev_out.value for input in interesting_tx.inputs]) / BITCOIN_TO_SATOSHI}")

    print_input_time_distribution(session, interesting_tx)


Transaction d508 has 955 vertices
It occured on 2011-07-14 17:46:33
hash of interesting_tx: d50871077b83f7f2497a65c8ff00172c9bbfefd46cd2c4c258a2cccbad337d82
136273:29:6 12Df (43.55870704)
136273:29:2 1fbN (0.00000001)
136273:29:3 1JxH (48.70949313)
min_valued is coinbase? True
Total tx input value: 340.87444721
input times: [datetime.datetime(2011, 7, 5, 11, 41), datetime.datetime(2011, 6, 28, 6, 9, 26), datetime.datetime(2011, 7, 6, 11, 20, 33), datetime.datetime(2011, 6, 29, 15, 13, 28), datetime.datetime(2011, 7, 5, 11, 41), datetime.datetime(2011, 6, 30, 22, 24, 12), datetime.datetime(2011, 6, 29, 4, 17, 39), datetime.datetime(2011, 6, 30, 19, 40, 24), datetime.datetime(2011, 6, 29, 10, 51, 16), datetime.datetime(2011, 6, 29, 15, 0, 51), datetime.datetime(2011, 7, 8, 16, 38, 48), datetime.datetime(2011, 6, 28, 6, 9, 26), datetime.datetime(2011, 7, 1, 15, 51, 45), datetime.datetime(2011, 6, 30, 21, 42, 20), datetime.datetime(2011, 6, 28, 6, 9, 26), datetime.datetime(2011, 6, 29, 1, 

┏━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Time         ┃ Block    ┃ Tx        ┃ Value (BTC) ┃ Coinbase ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 2011-06-27   │ 133573   │ 02499e1d… │ 36.02562453 │   Yes    │
│ 2011-06-27   │ 133583   │ db34bb2e… │ 16.48641448 │   Yes    │
│ 2011-06-28   │ 133633   │ 6be22994… │  6.00000000 │    No    │
│ 2011-06-28   │ 133633   │ 451718e7… │  7.00000000 │    No    │
│ 2011-06-28   │ 133633   │ 77cd0daf… │ 10.77456211 │   Yes    │
│ 2011-06-28   │ 133644   │ bd158c04… │ 38.36630329 │   Yes    │
│ 2011-06-28   │ 133729   │ 4bf0f6dc… │  0.21756692 │   Yes    │
│ 2011-06-29   │ 133764   │ be247786… │  0.00000001 │   Yes    │
│ 2011-06-29   │ 133779   │ 01b59529… │ 17.86076149 │   Yes    │
│ 2011-06-29   │ 133786   │ f526d99a… │ 43.55870704 │   Yes    │
│ 2011-06-29   │ 133813   │ 9666a767… │ 23.80965672 │   Yes    │
│ 2011-06-29   │ 133831   │ 3e9a1c10… │ 24.74114838 │   Yes    │
│ 2011-06-29   │ 133860   │ 297299cf… │ 21.22349919 │   Yes    │
│ 2011-06-29   │ 133861   │ 5eb00e48… │ 48.70949313 │   Yes    │
│ 2011-06-29   │ 133884   │ 0e850f4a… │ 34.66492050 │   Yes    │
│ 2011-06-30   │ 133988   │ 65256870… │  0.13476571 │   Yes    │
│ 2011-06-30   │ 134086   │ 8ff94cd6… │  0.00000001 │   Yes    │
│ 2011-06-30   │ 134100   │ 0583e8d8… │  0.00000001 │   Yes    │
│ 2011-06-30   │ 134106   │ 89be7c02… │  0.00000001 │   Yes    │
│ 2011-07-01   │ 134233   │ d235b338… │  0.00000001 │   Yes    │
│ 2011-07-05   │ 134863   │ 1c19389b… │ 11.12963614 │    No    │
│ 2011-07-05   │ 134863   │ 1718f403… │  0.00000001 │   Yes    │
│ 2011-07-06   │ 135016   │ a7c59f4c… │  0.00000001 │   Yes    │
│ 2011-07-06   │ 135075   │ c568f1ec… │  0.00000001 │   Yes    │
│ 2011-07-07   │ 135169   │ 18b27708… │  0.04208580 │   Yes    │
│ 2011-07-08   │ 135330   │ 99125fd5… │  0.12930170 │   Yes    │
└──────────────┴──────────┴───────────┴─────────────┴──────────┘

### Analyze Transaction `67e1...` with ID `1441873` (3rd Largest Tx from Blocks 0-200,000)

In [5]:
tx_key = '67e1'

display(visualize_graph(largest_txs[tx_key]['graph'], layout='dagre', node_color_map=largest_txs[tx_key]['color_map']))

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeDimensionsIncludeLabels': True, 'rankDir': 'LR'}, cyto…


### Analyze Transaction `41cb9d...` (4th Largest Tx from Blocks 0-200,000)

In [19]:
tx_key = '41cb'

display(visualize_graph(largest_txs[tx_key]['graph'], layout='dagre', node_color_map=largest_txs[tx_key]['color_map']))

### Analyze Transaction `2f66...` (5th Largest Tx from Blocks 0-200,000)

In [1]:
tx_key = '2f66'

display(visualize_graph(largest_txs[tx_key]['graph'], layout='dagre', node_color_map=largest_txs[tx_key]['color_map']))

NameError: name 'visualize_graph' is not defined

### Traverse Forward

In [ ]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

# interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'
# interesting_addr = '12cbQLTFMXRnSzktFkuoG3eHoMeFtpTu3S'
# interesting_addr = '1BBz9Z15YpELQ4QP5sEKb1SwxkcmPb5TMs'
# interesting_addr = '1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'
# interesting_addr = '1DUDsfc23Dv9sPMEk5RsrtfzCw5ofi5sVW'
# interesting_addr = '1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi'
interesting_addr = '13HtsYzne8xVPdGDnmJX8gHgBZerAfJGEf'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()
    
if not address:
    print(f"address {interesting_addr} not found")
    sys.exit(1)

print(f"id of address {address.addr:4}: {address.id}")

my_path = analyzer.get_vertex_path(address.id, 'address')
graph = analyzer.traversal_to_networkx(my_path, include_data=True)

print(graph)

coin_traces = analyzer.get_coin_traces(address.id, 'address', 'outgoing', graph, pretty_labels=True)

for source in coin_traces.values():
    print(f"amount from {source['label']} is {source['amount']}")

from ipycytoscape_graph_visualization import visualize_graph

display(visualize_graph(graph, layout='dagre'))